In [ ]:
import numpy as np
import tensorflow as tf
import src.model as modelfunc

In [ ]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:", tf.config.list_physical_devices('GPU'))


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Running on GPU")
else:
    print("GPU not available. Running on CPU")

## Load data

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model parameters
timesteps = None # Variable-length sequences
features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
num_classes = 5  # Actions (e.g., left turn, right turn, stopping)

# Simulated variable-length IMU data
sequences = [np.random.rand(np.random.randint(1000, 2000), features) for _ in range(100)]
labels = [np.random.randint(0, num_classes, len(seq)) for seq in sequences]  # One label per timestep

# Pad sequences
padded_sequences = pad_sequences(sequences, padding='post', dtype='float32')
padded_labels = pad_sequences(labels, padding='post', dtype='int32')

# One-hot encode labels
padded_labels = tf.keras.utils.to_categorical(padded_labels, num_classes)

padded_sequences = padded_sequences.astype('float32')
padded_labels = padded_labels.astype('float32')

# Check shapes
print("Padded sequences shape:", padded_sequences.shape)
print("Padded labels shape:", padded_labels.shape)
print("Padded sequences dtype:", padded_sequences.dtype)  # Should be float32
print("Padded labels dtype:", padded_labels.dtype)        # Should be float32


In [4]:
# Generate sample weights to ignore padded timesteps
sample_weights = np.array([
    [1 if np.any(timestep != 0) else 0 for timestep in sequence]
    for sequence in padded_sequences
])

## Build Model

In [ ]:
# Model parameters
timesteps = None # Variable-length sequences
features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
num_classes = 5  # Actions (e.g., left turn, right turn, stopping)

# Build the model
model = modelfunc.build_seq2seq_lstm((timesteps, features), num_classes)
model.summary()

## Train Model

In [ ]:
# Example call to the function
history = modelfunc.train_model(
    model=model,  # Your pre-defined Keras model
    X_train=padded_sequences, 
    y_train=padded_labels,
    sample_weight=sample_weights,  # Optional; pass None if not using sample weights
    batch_size=16,  # Optional; defaults to 16
    epochs=10       # Optional; defaults to 10
)


## Model prediction

In [ ]:
# Simulated test data
test_sequence = np.random.rand(80, features)  # Variable-length sequence
test_sequence_padded = pad_sequences([test_sequence], maxlen=padded_sequences.shape[1], padding='post')

# Make predictions
predictions = model.predict(test_sequence_padded)

# Get the confidence scores and the predicted classes
confidence_scores = np.max(predictions, axis=-1)  # Maximum probability for each timestep
predicted_classes = np.argmax(predictions, axis=-1)  # Class with the highest probability for each timestep

# Print results
print("Confidence scores shape:", confidence_scores.shape)  # Same shape as the input sequence
print("Predicted classes shape:", predicted_classes.shape)

# Example: For a specific sequence
example_idx = 0  # Index of the sequence to inspect
print(f"Sequence {example_idx}:")
print("Predicted classes:", predicted_classes[example_idx])
print("Confidence scores:", confidence_scores[example_idx])